# Blue Carbon Cost Tool Prototype

This notebooks prototypes the workflow for the Blue Carbon Cost tool. The tool is a excel base tool "[Blue Carbon Cost Tool - Excel full functionality (only if needed).xlsm](https://docs.google.com/spreadsheets/d/1lcmLolkwedgDrJJ54VvlMy3dn40rKjFi/edit?usp=sharing&ouid=102705365484464103073&rtpof=true&sd=true)" that allows the user to estimate the cost of a blue carbon project. The tool has been documented in the [2305 - Blue Carbon Tool - Guidance Document_vBetaTest (1)](https://docs.google.com/document/d/1lvktavnyMVnzBpULrCF3sg5eah6xA-PS/edit?usp=sharing&ouid=102705365484464103073&rtpof=true&sd=true).

For the current prototype we will use the "[Carbon-Cost Data Upload.xlsm](https://docs.google.com/spreadsheets/d/170jYrXpTSUlQfsMXXjN5v6L2PIlMYUXu/edit?usp=sharing&ouid=102705365484464103073&rtpof=true&sd=true)" file that has been manually simplified to only contain the data upload and the cost estimation. To reproduce this notebook you need to add the "master table" to this excel file in your local. You can copy the "master table" from these other excel file "[data_ingestion.xlsm](https://docs.google.com/spreadsheets/d/1BOadf1aM7SAjZcVYcvpljsPoNq0NOTHb/edit?usp=drive_link&ouid=102705365484464103073&rtpof=true&sd=true)".

## Setup

### Library import

In [1]:
import json
import sys

import pandas as pd  # noqa: I001

sys.path.append("../src/bcc_model/")

from blue_carbon_project import BlueCarbonProject
from breakeven_cost_calculator import BreakevenCostCalculator
from cost_calculator import CostCalculator

## Blue Carbon Cost Tool 

### Read data

In [2]:
# Import excel with the import datai
data_path = "../excel/Carbon-Cost Data Upload.xlsm"

**Create the master table**

To replicate the prototype we need to crete the `master_table` that the back-end team creates when the Excel is ingested. This table contains all the data contained in the `Cost` and `Carbon` tables of the Excel file.

In [3]:
index = pd.read_excel(data_path, sheet_name="Index", header=11)
index = index.iloc[:, 2:]
cost_tables = index[index["Type"] == "Cost Tables"]["Sheet Name"].values[1:]
carbon_tables = index[index["Type"] == "Carbon Tables"]["Sheet Name"].values[1:]
tables = list(cost_tables[1:]) + list(carbon_tables)

In [4]:
master_table = pd.read_excel(data_path, sheet_name=cost_tables[0]).drop(columns=["Source"])
for table in tables:
    # Read the table and drop the source columns
    df_table = pd.read_excel(data_path, sheet_name=table)
    df_table = df_table.drop(
        columns=[col for col in df_table.columns if col.startswith("Source")],
        errors='ignore'
        )
    # Determine which columns exist in df_table from the list
    common_columns = [
        col for col in [
            "Country", "Country code", "Activity", "Ecosystem"
            ] if col in df_table.columns
        ]

    # Merge using only the found columns
    master_table = pd.merge(master_table, df_table, on=common_columns, how='left')

# Convert master_table's column names to lower case and replace spaces with underscores
master_table.columns = master_table.columns.str.lower().str.replace(" - ", "_")\
    .str.replace("-", "_").str.replace(" / ", "_").str.replace(" ", "_")

master_table.head()

,country,country_code,activity,ecosystem,size_ha,feasibility_analysis_cost,planning_and_admin_cost,data_collection_and_field_cost,community_representation_liaison_cost,blue_carbon_project_planning_cost,...,mrv_cost,long_term_project_operating_cost,other_community_cash_flow,extent,extent_historic,unprotected_extent,loss_rate,restorable_land,tier_1_ipcc_default_value,tier_2_country_specific_rate
0,United States,USA,Restoration,Mangrove,500,100000,166766.666667,26666.666667,126500.000000,100000,...,100000,130600,Non-development,42057.401711,231272.524303,20077.3163,0,9665.780512,5.94,17.2349
1,Indonesia,IDN,Restoration,Mangrove,500,50000,166766.666667,26666.666667,71183.333333,100000,...,75000,26400,Development,2901690.307204,2930508.086034,2028220.9615,0,204536.1679,5.94,29.5
2,Australia,AUS,Restoration,Mangrove,500,70000,166766.666667,26666.666667,113016.666667,115000,...,100000,105800,Non-development,1004148.845873,1013552.135747,482831.9666,0,69160.39725,5.94,7.285
3,The Bahamas,BHS,Restoration,Mangrove,500,70000,166766.666667,26666.666667,103100.000000,125000,...,100000,87500,Non-development,150839.082513,144788.58522,34035.8866,0,9842.169833,5.94,7.5
4,Kenya,KEN,Restoration,Mangrove,500,50000,166766.666667,26666.666667,66050.000000,100000,...,75000,17700,Development,54121.500576,54080.010314,44742.4502,0,1609.6981,5.94,19


**Read other tables**

In [5]:
# Open the excel file - tab 'Base_size_table'
base_size = pd.read_excel(data_path, sheet_name="Base_size_table")

# Open the excel file - tab 'Base_increase'
base_increase = pd.read_excel(data_path, sheet_name="Base_increase")

base_size.head()

,id,activity,ecosystem,feasibility_analysis,conservation_planning_and_admin,data_collection_and_field_cost,community_representation,blue_carbon_project_planning,establishing_carbon_rights,financing_cost,validation,implementation_labor_planting,implementation_labor_hybrid,implementation_labor_hydrology,monitoring,baseline_reassessment,MRV,long_term_project_operating_cost
0,1,Restoration,Mangrove,500,2500,500,2500,500,500,500,500,0,0,0,2500,500,500,2500
1,2,Conservation,Mangrove,20000,20000,20000,20000,20000,20000,20000,20000,0,0,0,20000,20000,20000,20000
2,3,Restoration,Seagrass,500,2500,500,2500,500,500,500,500,0,0,0,2500,500,500,2500
3,4,Conservation,Seagrass,20000,20000,20000,20000,20000,20000,20000,20000,0,0,0,20000,20000,20000,20000
4,5,Restoration,Salt marsh,500,2500,500,2500,500,500,500,500,0,0,0,2500,500,500,2500


### Create example projects

In this section we will replicate the steps followed in the `Dashboard` sheet of the [`Blue Carbon Cost Tool - Excel full functionality (only if needed)`](https://docs.google.com/spreadsheets/d/1lcmLolkwedgDrJJ54VvlMy3dn40rKjFi/edit?usp=sharing&ouid=102705365484464103073&rtpof=true&sd=true) Excel file following the `Deep dive: Excel dashboard` section in the [`Blue Carbon Tool - Guidance Document`](https://docs.google.com/document/d/1lvktavnyMVnzBpULrCF3sg5eah6xA-PS/edit?usp=sharing&ouid=102705365484464103073&rtpof=true&sd=true).

**(1) Input - Project parameters**

This section outlines the standard project parameters for a hypothetical blue carbon project. Users have the flexibility to adjust these parameters according to their preferences or specific requirements.

In [ ]:
## EXAMPLE 1

# project_name = "Conservation_Mangrove_Indonesia"
# loss_rate_used = "project-specific"
# # Example usage for a restoration project
# Project = BlueCarbonProject(
#     activity="Conservation",  # ['Restoration', 'Conservation']
#     ecosystem="Mangrove",  #  ['Mangrove', 'Seagrass', 'Salt marsh']
#     country="Indonesia",  # [
#     #'United States', 'Indonesia', 'Australia', 'The Bahamas', 'Kenya', 'Mexico',
#     # 'Colombia', 'India', 'China']
#     master_table=master_table,
#     base_size=base_size,
#     base_increase=base_increase,
#     carbon_price=20,  # Default value 30
#     carbon_revenues_to_cover="Opex",  # ['Opex', 'capex+Opex']
#     project_size_ha=10000,
#     # restoration_activity='Planting', # ['Planting', 'Hybrid', 'Hydrology']
#     # sequestration_rate_used='Tier 1 - IPCC default value', # ['Tier 1 - IPCC default value',
#     #  'Tier 2 - Country-specific rate', 'Tier 3 - Project-specific rate']
#     # project_specific_sequestration_rate=None,
#     # planting_success_rate=0.8, # Default value 0.8
#     loss_rate_used="project-specific",  # ['National average', 'project-specific']
#     project_specific_loss_rate=-0.001, # -0.10% loss rate
#     emission_factor_used="Tier 2 - Country-specific emission factor",
#     # ['Tier 1 - Global emission factor', 'Tier 2 - Country-specific emission factor',
#     #  'Tier 3 - Project specific emission factor']
#     # tier_3_project_specific_emission="AGB and SOC separately",
#     # ['One emission factor', 'AGB and SOC separately']
#     # tier_3_project_specific_emission_one_factor=0.5,
#     # tier_3_emission_factor_AGB=0.5,
#     # tier_3_emission_factor_SOC=0.5
# )

In [ ]:
# ### EXAMPLE 2

# project_name = "Restoration__China_Salt_marsh"
# loss_rate_used = "project-specific"
# # Example usage for a restoration project
# Project = BlueCarbonProject(
#     activity="Restoration",  # ['Restoration', 'Conservation']
#     ecosystem="Salt marsh",  #  ['Mangrove', 'Seagrass', 'Salt marsh']
#     country="China",  # [
#     #'United States', 'Indonesia', 'Australia', 'The Bahamas', 'Kenya', 'Mexico',
#     # 'Colombia', 'India', 'China']
#     master_table=master_table,
#     base_size=base_size,
#     base_increase=base_increase,
#     carbon_price=30,  # Default value 30
#     carbon_revenues_to_cover="Opex",  # ['Opex', 'capex+Opex']
#     project_size_ha=500,
#     restoration_activity='Hydrology', # ['Planting', 'Hybrid', 'Hydrology']
#     sequestration_rate_used='Tier 1 - IPCC default value', # ['Tier 1 - IPCC default value',
#     #  'Tier 2 - Country-specific rate', 'Tier 3 - Project-specific rate']
#     # project_specific_sequestration_rate=None,
#     planting_success_rate=0.8, # Default value 0.8
#     # loss_rate_used="project-specific",  # ['National average', 'project-specific']
#     # project_specific_loss_rate=-0.001, # -0.10% loss rate
#     # emission_factor_used="Tier 2 - Country-specific emission factor",
#     # ['Tier 1 - Global emission factor', 'Tier 2 - Country-specific emission factor',
#     #  'Tier 3 - Project specific emission factor']
#     # tier_3_project_specific_emission="AGB and SOC separately",
#     # ['One emission factor', 'AGB and SOC separately']
#     # tier_3_project_specific_emission_one_factor=0.5,
#     # tier_3_emission_factor_AGB=0.5,
#     # tier_3_emission_factor_SOC=0.5
# )

In [6]:
### EXAMPLE 3

project_name = "Restoration_Mexico_Mangrove"
# loss_rate_used = "project-specific"
# Example usage for a restoration project
Project = BlueCarbonProject(
    activity="Restoration",  # ['Restoration', 'Conservation']
    ecosystem="Mangrove",  #  ['Mangrove', 'Seagrass', 'Salt marsh']
    country="Mexico",  # [
    #'United States', 'Indonesia', 'Australia', 'The Bahamas', 'Kenya', 'Mexico',
    # 'Colombia', 'India', 'China']
    master_table=master_table,
    base_size=base_size,
    base_increase=base_increase,
    carbon_price=30,  # Default value 30
    carbon_revenues_to_cover="capex+Opex",  # ['Opex', 'capex+Opex']
    project_size_ha=500,
    restoration_activity="Planting",  # ['Planting', 'Hybrid', 'Hydrology']
    sequestration_rate_used="Tier 2 - Country-specific rate",  # ['Tier 1 - IPCC default value',
    #  'Tier 2 - Country-specific rate', 'Tier 3 - Project-specific rate']
    # project_specific_sequestration_rate=None,
    planting_success_rate=0.8,  # Default value 0.8
    # loss_rate_used="project-specific",  # ['National average', 'project-specific']
    # project_specific_loss_rate=-0.001, # -0.10% loss rate
    # emission_factor_used="Tier 2 - Country-specific emission factor",
    # ['Tier 1 - Global emission factor', 'Tier 2 - Country-specific emission factor',
    #  'Tier 3 - Project specific emission factor']
    # tier_3_project_specific_emission="AGB and SOC separately",
    # ['One emission factor', 'AGB and SOC separately']
    # tier_3_project_specific_emission_one_factor=0.5,
    # tier_3_emission_factor_AGB=0.5,
    # tier_3_emission_factor_SOC=0.5
)

**(2) Input - Model assumptions** 

Users can modify model assumptions by overriding the standard base values for conducting bespoke project analysis.

In [ ]:
# Project.set_additional_assumptions(
#     # verification_frequency,
#     # discount_rate,
#     # carbon_price_increase,
#     buffer=0.28,
#     # baseline_reassessment_frequency,
#     conservation_project_length=30,
#     # restoration_project_length,
#     # restoration_rate
# )

**(3) Input - Cost components**

The model always displays these standard costs, but users have the option to override them for customized project analysis.


In [ ]:
# Project.override_cost_input(
#     feasibility_analysis=30000
#     # conservation_planning_and_admin,
#     # data_collection_and_field_cost,
#     # community_representation,
#     # blue_carbon_project_planning,
#     # establishing_carbon_rights,
#     # validation,
#     # implementation_labor,
#     # monitoring,
#     # maintenance,
#     # community_benefit_sharing_fund,
#     # carbon_standard_fees,
#     # baseline_reassessment,
#     # MRV,
#     # long_term_project_operating,
#     # financing_cost
# )

**(4) Input – Restoration plan** 

Used for restoration projects only. Users can use this section to have more control over the restoration plan input.
To have greater control over the timeline of the restoration plan, kindly provide the annual hectares restored in the blue space provided. Ensure that the cumulative sum does not exceed your original project size and remains within your designated project timeline. 

In [7]:
restoration_plan = {}
restoration_plan[-1] = 250
restoration_plan.update(dict.fromkeys(range(1, 31), 5))
cumulative_size = Project.calculate_total_restoration()
if cumulative_size <= Project.project_size_ha:
    Project.update_restoration_plan(restoration_plan)

**Check all input parameters**

In [8]:
project_parameters = Project.get_project_parameters()

### Project Parameters
| Parameter                                      | Value                                   |
| ---------------------------------------------- | --------------------------------------- |
| Project size (ha) | 500 |
| Initial carbon price assumption ($) | 30 |
| Country | Mexico |
| Ecosystem | Mangrove |
| Activity | Restoration |
| Carbon revenues to cover | capex+Opex |
| Carbon revenues will not cover | None |
**Additional parameters for a Restoration project**
| Activity type - Restoration | Planting |
| Sequestration rate used | Tier 2 - Country-specific rate |
| Tier 2 - Country-specific rate | 18.33333 |
| Project-specific sequestration rate (tCO2e/ha/yr) | None |
| Planting Success Rate (%) | 80.0 |
### Assumptions
| Parameter                                      | Value                                   |
| ---------------------------------------------- | --------------------------------------- |
| Verification frequency (yr) | 5 |
| Discount rate (%) | 4.0 |
| Carbon price increase (%) | 1.5 |
| Buffer (%) | 20 |
| Baseline reassessment frequency (yr) | 10 |
**Additional assumptions for a Restoration project**
| Restoration project length (yr) | 20 |
| Restoration rate (ha/yr) | 250 |
### Cost Inputs
| Parameter                                      | Value                                   |
| ---------------------------------------------- | --------------------------------------- |
| **CAPEX**                                      |                                         |
| Feasibility_analysis ($/project) | 50,000 |
| Conservation planning and admin ($/yr) | 166,767 |
| Data collection and field cost ($/yr) | 26,667 |
| Community representation ($/yr) | 72,600 |
| Blue carbon project planning ($/project) | 100,000 |
| Establishing carbon rights ($/yr) | 100,000 |
| Validation ($/project) | 50,000 |
| Implementation labor ($/ha) | 15,986 |
| **OPEX**                                       |                                         |
| Monitoring ($/yr) | 11,900 |
| Maintenance (% of implementation labor) | 8 |
| Community benefit sharing fund (% of revenue) | 50 |
| Carbon standard fees ($/credit) | 0.20 |
| Baseline reassessment ($/event) | 40,000 |
| MRV ($/event) | 75,000 |
| Long-term project operating ($/yr) | 31,300 |
| **Other**                                       |                                         |
| Financing cost (% of capex) | 5 |
### Restoration Plan
| Year                                           | Annual ha                               |
| ---------------------------------------------- | --------------------------------------- |
| -1 | 250 |
| 1 | 5 |
| 2 | 5 |
| 3 | 5 |
| 4 | 5 |
| 5 | 5 |
| 6 | 5 |
| 7 | 5 |
| 8 | 5 |
| 9 | 5 |
| 10 | 5 |
| 11 | 5 |
| 12 | 5 |
| 13 | 5 |
| 14 | 5 |
| 15 | 5 |
| 16 | 5 |
| 17 | 5 |
| 18 | 5 |
| 19 | 5 |
| 20 | 5 |
| 21 | 5 |
| 22 | 5 |
| 23 | 5 |
| 24 | 5 |
| 25 | 5 |
| 26 | 5 |
| 27 | 5 |
| 28 | 5 |
| 29 | 5 |
| 30 | 5 |

### Calculate costs

In [9]:
cost_calculator = CostCalculator(Project)

**(5) Output - Project summary** 

This table provides a concise summary “snapshot” of the key economic outcomes for the hypothetical blue carbon project. 

In [10]:
cost_summary = cost_calculator.get_summary()

### Project Summary
| Parameter                            | Value                                             |
| ------------------------------------ | ------------------------------------------------- |
| Project | Mexico Mangrove Restoration (500 ha) |
| $/tCO2e (total cost, NPV) | $103 |
| $/ha | $23,676 |
| NPV covering cost | $-9,466,615 |
| IRR when priced to cover opex | -9.5% |
| IRR when priced to cover total costs | -19.2% |
| Total cost (NPV) | $11,837,930 |
| Capital expenditure (NPV) | $8,262,546 |
| Operating expenditure (NPV) | $3,575,384 |
| Credits issued | 114,400 |
| Total revenue (NPV) | $2,371,315 |
| Total revenue (non-discounted) | $4,041,491 |
| Financing cost | $468,523 |
| Funding gap (NPV) | $9,466,615 |
| Funding gap per tCO2e (NPV) | $82.8 |
| Community benefit sharing fund % of revenue | 50% |

**(6) Output - Detailed cost summary**

This table shows a comprehensive breakdown of each cost component for the hypothetical blue carbon project. 

In [11]:
cost_estimates = cost_calculator.get_cost_estimates()

### Cost Estimates
| Cost estimates (USD)   | Total cost      | NPV             |
| ---------------------- | --------------- | --------------- |
| **Capital expenditure** | **$9,370,467** | **$8,262,546** |
| Feasibility analysis | $50,000 | $50,000 |
| Conservation planning and admin | $667,067 | $629,559 |
| Data collection and field costs | $80,000 | $76,963 |
| Community representation / liaison | $290,400 | $274,072 |
| Blue carbon project planning | $100,000 | $88,900 |
| Establishing carbon rights | $140,000 | $129,504 |
| Validation | $50,000 | $44,450 |
| Implementation labor | $7,993,000 | $6,969,099 |
| **Operating expenditure** | **$5,325,076** | **$3,575,384** |
| Monitoring | $238,000 | $143,773 |
| Maintenance | $1,997,451 | $1,611,012 |
| Community benefit sharing fund | $2,020,746 | $1,185,658 |
| Carbon standard fees | $22,880 | $13,674 |
| Baseline reassessment | $120,000 | $75,812 |
| MRV | $300,000 | $167,296 |
| Long-term project operating | $626,000 | $378,159 |
| **Total cost** | **$14,695,543** | **$11,837,930** |

**(7) Output - Sensitivity analysis**

This section presents a sensitivity analysis, illustrating the relative impact of changes to cost components on the hypothetical blue carbon project. 

**(8) Output - Projected cash flow** 

This section shows the projected cash flow timing for the hypothetical blue carbon project. 

In [12]:
cost_graph = cost_calculator.plot_financial_data()
cost_graph

**“Pro Forma” sheet**

This section presents the yearly cash flows associated with the hypothetical blue carbon project.

In [13]:
yearly_cost_breakdown = cost_calculator.get_yearly_cost_breakdown()

### Pro Forma Financials
| Year | -4 | -3 | -2 | -1 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | Total | NPV |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| Feasibility analysis | -50000.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -50000.0 | -50000.0 |
| Conservation planning and admin | -166767.0 | -166767.0 | -166767.0 | -166767.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -667067.0 | -629559.0 |
| Data collection and field costs | -26667.0 | -26667.0 | -26667.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -80000.0 | -76963.0 |
| Community representation / liaison | -72600.0 | -72600.0 | -72600.0 | -72600.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -290400.0 | -274072.0 |
| Blue carbon project planning | 0 | 0 | 0 | -100000.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -100000.0 | -88900.0 |
| Establishing carbon rights | 0 | -46667.0 | -46667.0 | -46667.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -140000.0 | -129504.0 |
| Validation | 0 | 0 | 0 | -50000.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -50000.0 | -44450.0 |
| Implementation labor | 0 | 0 | 0 | -3996500.0 | -3996500.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -0.0 | -7993000.0 | -6969099.0 |
| **Total capex** | **-316033.0** | **-312700.0** | **-312700.0** | **-4432533.0** | **-3996500.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-0.0** | **-9370467.0** | **-8262546.0** |
| Monitoring | 0 | 0 | 0 | 0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -11900.0 | -238000.0 | -143773.0 |
| Maintenance | 0 | 0 | 0 | 0 | -332908.0 | -665817.0 | -665817.0 | -332908.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -1997451.0 | -1611012.0 |
| Community benefit sharing fund | -0.0 | -0.0 | -0.0 | -0.0 | -44660.0 | -90660.0 | -92020.0 | -93400.0 | -94801.0 | -96223.0 | -97666.0 | -99131.0 | -100618.0 | -102128.0 | -103659.0 | -105214.0 | -106793.0 | -108394.0 | -110020.0 | -111671.0 | -113346.0 | -115046.0 | -116772.0 | -118523.0 | -2020746.0 | -1185658.0 |
| Carbon standard fees | 0 | 0 | 0 | 0 | -587.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -1173.0 | -22880.0 | -13674.0 |
| Baseline reassessment | 0 | 0 | 0 | -40000.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -40000.0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -40000.0 | -120000.0 | -75812.0 |
| MRV | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | -75000.0 | 0 | 0 | 0 | 0 | -75000.0 | 0 | 0 | 0 | 0 | -75000.0 | 0 | 0 | 0 | 0 | -75000.0 | -300000.0 | -167296.0 |
| Long-term project operating | 0 | 0 | 0 | 0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -31300.0 | -626000.0 | -378159.0 |
| **Total opex** | **-0.0** | **-0.0** | **-0.0** | **-40000.0** | **-421355.0** | **-800850.0** | **-802210.0** | **-470682.0** | **-214174.0** | **-140596.0** | **-142040.0** | **-143505.0** | **-144992.0** | **-261501.0** | **-148033.0** | **-149588.0** | **-151166.0** | **-152768.0** | **-229394.0** | **-156044.0** | **-157719.0** | **-159419.0** | **-161145.0** | **-277897.0** | **-5325076.0** | **-3575384.0** |
| **Total cost** | **-316033.0** | **-312700.0** | **-312700.0** | **-4472533.0** | **-4417855.0** | **-800850.0** | **-802210.0** | **-470682.0** | **-214174.0** | **-140596.0** | **-142040.0** | **-143505.0** | **-144992.0** | **-261501.0** | **-148033.0** | **-149588.0** | **-151166.0** | **-152768.0** | **-229394.0** | **-156044.0** | **-157719.0** | **-159419.0** | **-161145.0** | **-277897.0** | **-14695543.0** | **-11837930.0** |
| Est. credits issued | 0 | 0 | 0 | 0.0 | 2933.33 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 5866.67 | 114399.98 | 0 |
| **Est. revenue** | **0** | **0** | **0** | **0.0** | **89320.0** | **181320.0** | **184039.0** | **186800.0** | **189602.0** | **192446.0** | **195333.0** | **198263.0** | **201237.0** | **204255.0** | **207319.0** | **210429.0** | **213585.0** | **216789.0** | **220041.0** | **223341.0** | **226692.0** | **230092.0** | **233543.0** | **237046.0** | **4041491.0** | **2371315.0** |
| Annual net cash flow | -316033.0 | -312700.0 | -312700.0 | -4472533.0 | -4328535.0 | -619530.0 | -618171.0 | -283882.0 | -24572.0 | 51850.0 | 53293.0 | 54758.0 | 56245.0 | -57246.0 | 59286.0 | 60841.0 | 62419.0 | 64021.0 | -9353.0 | 67297.0 | 68972.0 | 70673.0 | 72398.0 | -40850.0 | -10654052.0 | -9466615.0 |
| Annual net income (revenue – OPEX) | 0.0 | 0.0 | 0.0 | -40000.0 | -332035.0 | -619530.0 | -618171.0 | -283882.0 | -24572.0 | 51850.0 | 53293.0 | 54758.0 | 56245.0 | -57246.0 | 59286.0 | 60841.0 | 62419.0 | 64021.0 | -9353.0 | 67297.0 | 68972.0 | 70673.0 | 72398.0 | -40850.0 | -1283585.0 | -1204069.0 |
| Cumulative net income (revenue - OPEX) | 0.0 | 0.0 | 0.0 | -35560.0 | -319385.0 | -828594.0 | -1317143.0 | -1532870.0 | -1550825.0 | -1514396.0 | -1478393.0 | -1442823.0 | -1407693.0 | -1442073.0 | -1407837.0 | -1374054.0 | -1340728.0 | -1307861.0 | -1312478.0 | -1280536.0 | -1249057.0 | -1218044.0 | -1187495.0 | -1204069.0 | 0 | 0 |
| Cumulative net income (revenue - CAPEX - OPEX) | -316033.0 | -616706.0 | -905815.0 | -4881881.0 | -8581931.0 | -9091140.0 | -9579689.0 | -9795416.0 | -9813371.0 | -9776942.0 | -9740939.0 | -9705369.0 | -9670239.0 | -9704619.0 | -9670383.0 | -9636600.0 | -9603274.0 | -9570407.0 | -9575024.0 | -9543082.0 | -9511604.0 | -9480590.0 | -9450041.0 | -9466615.0 | 0 | 0 |

In [ ]:
# Export to test_data to compare calculated results with expected ones
# yearly_cost_breakdown.to_csv("./test_data/Pro_forma_calculated_data.csv", index=False)

### Calculate breakeven price

Calculate carbon price to obtain NPV covering cost to zero.

In [14]:
# Calculate the breakeven cost
breakeven_cost_calculator = BreakevenCostCalculator(Project)
breakeven_cost = breakeven_cost_calculator.calculate_breakeven_cost()
print(f"The breakeven cost is: {breakeven_cost["breakeven_carbon_price"]}")

Iteration 0: NPV covering cost = -9466615.116345167,
            Carbon price = 30
Iteration 1: NPV covering cost = -6196170.3445773125,
            Carbon price = 112.75014718136562
Iteration 2: NPV covering cost = -4055570.7047527824,
            Carbon price = 166.91248521592806
Iteration 3: NPV covering cost = -2654487.018040661,
            Carbon price = 202.36328453524106
Iteration 4: NPV covering cost = -1737437.6732450314,
            Carbon price = 225.56684660407632
Iteration 5: NPV covering cost = -1137202.6489092708,
            Carbon price = 240.75424161406625
Iteration 6: NPV covering cost = -744331.6583960578,
            Carbon price = 250.694826016806
Iteration 7: NPV covering cost = -487186.35875673965,
            Carbon price = 257.2012228151374
Iteration 8: NPV covering cost = -318877.4056313932,
            Carbon price = 261.4598456065368
Iteration 9: NPV covering cost = -208714.38207280263,
            Carbon price = 264.2472360227004
Iteration 10: NPV coverin

### Export outputs as `JSON` file

In [ ]:
# populate json with output values
output = {
    "initialCarbonPriceComputationOutput": {
        # Update for restoration projects and other definitions
        # "lossRate": project_parameters["Project parameters"][
        #    "For Conservation Projects Only"
        #    ]["Loss rate used"],
        "sequestrationRate": project_parameters["Project parameters"][
            "For Restoration Projects Only"
            ]["Tier 2 - Country-specific rate"],
        "plantingSuccessRate": 0.8,
        "carbonRevenuesToCover": project_parameters["Project parameters"][
            "Carbon revenues to cover"
            ],
        "initialCarbonPrice": project_parameters["Project parameters"][
            "Initial carbon price assumption ($)"
            ],
        # "emissionFactors": {
        #     "emissionFactor": None,
        #     "emissionFactorAgb": project_parameters["Project parameters"]['For Conservation Projects Only']['Country-specific emission factors']['Emission factor AGB'],  # noqa: E501
        #     "emissionFactorSoc": project_parameters["Project parameters"]['For Conservation Projects Only']['Country-specific emission factors']['Emission factor SOC']  # noqa: E501
        # },
        "totalProjectCost": {
            "total": {
                "total": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "capex": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Capital expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
            "npv": {
                "total": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "capex": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Capital expenditure", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
        },
        "leftover": {
            # review when is OPEX + CAPEX
            "total": {
                "total": int(
                    cost_summary["Project summary"][
                        "Total revenue (non-discounted)"
                        ].replace("$", "").replace(",", "")
                ),
                "leftover": int(
                    cost_summary["Project summary"][
                        "Total revenue (non-discounted)"
                        ].replace("$", "").replace(",", "")
                )
                - int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if project_parameters["Project parameters"]["Carbon revenues to cover"] == "Opex"
                else int(
                    cost_summary["Project summary"][
                        "Total revenue (non-discounted)"
                        ].replace("$", "").replace(",", "")
                )
                - int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if project_parameters["Project parameters"]["Carbon revenues to cover"] == "Opex"
                else int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
            "npv": {
                "total": int(cost_summary["Project summary"][
                    "Total revenue (NPV)"
                    ].replace("$", "").replace(",", "")),
                "leftover": int(
                    cost_summary["Project summary"][
                        "Total revenue (NPV)"
                        ].replace("$", "").replace(",", "")
                )
                - int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if project_parameters["Project parameters"]["Carbon revenues to cover"] == "Opex"
                else int(cost_summary["Project summary"][
                    "Total revenue (NPV)"
                    ].replace("$", "").replace(",", ""))
                - int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if project_parameters["Project parameters"]["Carbon revenues to cover"] == "Opex"
                else int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
        },
        "summary": {
            "$/tCO2e (total cost, NPV)": int(
                cost_summary["Project summary"][
                    "$/tCO2e (total cost, NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "$/ha": int(cost_summary["Project summary"]["$/ha"].replace("$", "").replace(",", "")),
            "NPV covering cost": int(
                cost_summary["Project summary"][
                    "NPV covering cost"
                    ].replace("$", "").replace(",", "")
            ),
            # double check this attribute - this value should be the same as the NPV covering cost so how is it used?  # noqa: E501
            "Leftover after OpEx / total cost": int(
                cost_summary["Project summary"][
                    "NPV covering cost"
                    ].replace("$", "").replace(",", "")
            ),
            "IRR when priced to cover OpEx": float(
                cost_summary["Project summary"]["IRR when priced to cover opex"].replace("%", "")
            )
            / 100,
            "IRR when priced to cover total cost": float(
                cost_summary["Project summary"][
                    "IRR when priced to cover total costs"
                    ].replace("%", "")
            )
            / 100,
            "Total cost (NPV)": int(
                cost_summary["Project summary"][
                    "Total cost (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Capital expenditure (NPV)": int(
                cost_summary["Project summary"][
                    "Capital expenditure (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Operating expenditure (NPV)": int(
                cost_summary["Project summary"][
                    "Operating expenditure (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Credits issued": int(cost_summary["Project summary"][
                "Credits issued"
                ].replace(",", "")),
            "Total revenue (NPV)": int(
                cost_summary["Project summary"][
                    "Total revenue (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Total revenue (non-discounted)": int(
                cost_summary["Project summary"][
                    "Total revenue (non-discounted)"
                    ].replace("$", "").replace(",", "")
            ),
            "Financing cost": int(cost_summary["Project summary"][
                "Financing cost"
                ].replace("$", "").replace(",", "")),
            # dueble check - we should remove this and just show the fundin gap (NPV)
            "Funding gap": int(cost_summary["Project summary"][
                "Funding gap (NPV)"
                ].replace("$", "").replace(",", "")),
            "Funding gap (NPV)": int(
                cost_summary["Project summary"][
                    "Funding gap (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Funding gap per tCO2e (NPV)": float(
                cost_summary["Project summary"][
                    "Funding gap per tCO2e (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Community benefit sharing fund": int(
                cost_summary["Project summary"]["Community benefit sharing fund % of revenue"]
                .replace("%", "")
                .replace(",", "")
            )
            / 100,
        },
        "costDetails": {
            "total": {
                "capitalExpenditure": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Capital expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "operationalExpenditure": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "totalCost": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "feasibilityAnalysis": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Feasibility analysis",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "conservationPlanningAndAdmin": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Conservation planning and admin",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "dataCollectionAndFieldCost": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Data collection and field costs",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityRepresentation": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"]
                        == "Community representation / liaison",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "blueCarbonProjectPlanning": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Blue carbon project planning",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "establishingCarbonRights": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Establishing carbon rights",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "validation": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Validation", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "implementationLabor": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Implementation labor",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "monitoring": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Monitoring", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "maintenance": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Maintenance", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityBenefitSharingFund": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Community benefit sharing fund",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "carbonStandardFees": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Carbon standard fees",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "baselineReassessment": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Baseline reassessment",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "mrv": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "MRV", "Total cost"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "longTermProjectOperatingCost": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Long-term project operating",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
            "npv": {
                "capitalExpenditure": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Capital expenditure", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "operationalExpenditure": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Operating expenditure", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "totalCost": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Total cost", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "feasibilityAnalysis": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Feasibility analysis", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "conservationPlanningAndAdmin": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Conservation planning and admin",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "dataCollectionAndFieldCost": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Data collection and field costs",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityRepresentation": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"]
                        == "Community representation / liaison",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "blueCarbonProjectPlanning": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Blue carbon project planning",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "establishingCarbonRights": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Establishing carbon rights",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "validation": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Validation", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "implementationLabor": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Implementation labor", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "monitoring": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Monitoring", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "maintenance": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Maintenance", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityBenefitSharingFund": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Community benefit sharing fund",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "carbonStandardFees": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Carbon standard fees", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "baselineReassessment": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Baseline reassessment", "NPV"
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "mrv": int(
                    cost_estimates.loc[cost_estimates["Cost estimates (USD)"] == "MRV", "NPV"]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "longTermProjectOperatingCost": int(
                    cost_estimates.loc[
                        cost_estimates["Cost estimates (USD)"] == "Long-term project operating",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
        },
        "yearlyBreakdown": [
            {
                "costName": "feasibilityAnalysis",
                "totalCost": float(yearly_cost_breakdown.loc["Feasibility analysis"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Feasibility analysis"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Feasibility analysis"][:-2].items()),
            },
            {
                "costName": "conservationPlanningAndAdmin",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Conservation planning and admin"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Conservation planning and admin"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Conservation planning and admin"
                    ][:-2].items()),
            },
            {
                "costName": "dataCollectionAndFieldCost",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Data collection and field costs"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Data collection and field costs"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Data collection and field costs"
                    ][:-2].items()),
            },
            {
                "costName": "blueCarbonProjectPlanning",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Blue carbon project planning"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Blue carbon project planning"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Blue carbon project planning"
                    ][:-2].items()),
            },
            {
                "costName": "communityRepresentation",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Community representation / liaison"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Community representation / liaison"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Community representation / liaison"
                    ][:-2].items()),
            },
            {
                "costName": "establishingCarbonRights",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Establishing carbon rights"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Establishing carbon rights"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Establishing carbon rights"
                    ][:-2].items()),
            },
            {
                "costName": "validation",
                "totalCost": float(yearly_cost_breakdown.loc["Validation"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Validation"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Validation"][:-2].items()),
            },
            {
                "costName": "implementationLabor",
                "totalCost": float(yearly_cost_breakdown.loc["Implementation labor"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Implementation labor"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Implementation labor"][:-2].items()),
            },
            {
                "costName": "monitoring",
                "totalCost": float(yearly_cost_breakdown.loc["Monitoring"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Monitoring"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Monitoring"][:-2].items()),
            },
            {
                "costName": "maintenance",
                "totalCost": float(yearly_cost_breakdown.loc["Maintenance"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Maintenance"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Maintenance"][:-2].items()),
            },
            {
                "costName": "communityBenefitSharingFund",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Community benefit sharing fund"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Community benefit sharing fund"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Community benefit sharing fund"
                    ][:-2].items()),
            },
            {
                "costName": "carbonStandardFees",
                "totalCost": float(yearly_cost_breakdown.loc["Carbon standard fees"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Carbon standard fees"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Carbon standard fees"][:-2].items()),
            },
            {
                "costName": "baselineReassessment",
                "totalCost": float(yearly_cost_breakdown.loc["Baseline reassessment"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Baseline reassessment"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Baseline reassessment"][:-2].items()),
            },
            {
                "costName": "mrv",
                "totalCost": float(yearly_cost_breakdown.loc["MRV"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["MRV"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["MRV"][:-2].items()),
            },
            {
                "costName": "longTermProjectOperatingCost",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Long-term project operating"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Long-term project operating"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Long-term project operating"
                    ][:-2].items()),
            },
            {
                "costName": "opexTotalCostPlan",
                "totalCost": float(yearly_cost_breakdown.loc["Total opex"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Total opex"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Total opex"][:-2].items()),
            },
            {
                "costName": "capexTotalCostPlan",
                "totalCost": float(yearly_cost_breakdown.loc["Total capex"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Total capex"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Total capex"][:-2].items()),
            },
            {
                "costName": "totalCostPlan",
                "totalCost": float(yearly_cost_breakdown.loc["Total cost"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Total cost"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Total cost"][:-2].items()),
            },
            {
                "costName": "estimatedRevenuePlan",
                "totalCost": float(yearly_cost_breakdown.loc["Est. revenue"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Est. revenue"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Est. revenue"][:-2].items()),
            },
            {
                "costName": "creditsIssuedPlan",
                "totalCost": float(yearly_cost_breakdown.loc["Est. credits issued"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Est. credits issued"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Est. credits issued"][:-2].items()),
            },
            {
                "costName": "cumulativeNetIncomePlan",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Cumulative net income (revenue - OPEX)"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Cumulative net income (revenue - OPEX)"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Cumulative net income (revenue - OPEX)"
                    ][:-2].items()),
            },
            {
                "costName": "cumulativeNetIncomeCapexOpex",
                "totalCost": float(
                    yearly_cost_breakdown.loc[
                        "Cumulative net income (revenue - CAPEX - OPEX)"
                        ]["Total"]
                ),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Cumulative net income (revenue - CAPEX - OPEX)"
                    ]["NPV"]),
                "costValues": dict(
                    yearly_cost_breakdown.loc[
                        "Cumulative net income (revenue - CAPEX - OPEX)"
                        ][:-2].items()
                ),
            },
            {
                "costName": "annualNetCashFlow",
                "totalCost": float(yearly_cost_breakdown.loc["Annual net cash flow"]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc["Annual net cash flow"]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc["Annual net cash flow"][:-2].items()),
            },
            {
                "costName": "annualNetIncome",
                "totalCost": float(yearly_cost_breakdown.loc[
                    "Annual net income (revenue – OPEX)"
                    ]["Total"]),
                "totalNPV": float(yearly_cost_breakdown.loc[
                    "Annual net income (revenue – OPEX)"
                    ]["NPV"]),
                "costValues": dict(yearly_cost_breakdown.loc[
                    "Annual net income (revenue – OPEX)"
                    ][:-2].items()),
            },
        ],
    },
    "breakevenPriceComputationOutput": {
        # Update values as first element of the list
        # "lossRate": breakeven_cost['project_params']["Project parameters"]['For Conservation Projects Only']['project-specific'],  # noqa: E501
        "carbonRevenuesToCover": breakeven_cost["project_params"][
            "Project parameters"
            ]["Carbon revenues to cover"],
        "sequestrationRate": project_parameters["Project parameters"][
            "For Restoration Projects Only"
            ]["Tier 2 - Country-specific rate"],
        "plantingSuccessRate": 0.8,
        "initialCarbonPrice": float(
            breakeven_cost["project_params"][
                "Project parameters"
                ]["Initial carbon price assumption ($)"]
        ),
        # "emissionFactors": {
        #     "emissionFactor": None,
        #     "emissionFactorAgb": breakeven_cost['project_params']["Project parameters"]['For Conservation Projects Only']['Country-specific emission factors']['Emission factor AGB'],  # noqa: E501
        #     "emissionFactorSoc": breakeven_cost['project_params']["Project parameters"]['For Conservation Projects Only']['Country-specific emission factors']['Emission factor SOC']  # noqa: E501
        # },
        "totalProjectCost": {
            "total": {
                "total": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "capex": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Capital expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
            "npv": {
                "total": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "capex": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Capital expenditure",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
        },
        "leftover": {
            "total": {
                "total": int(
                    breakeven_cost["cost_summary"][
                        "Project summary"
                        ]["Total revenue (non-discounted)"]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "leftover": int(
                    breakeven_cost["cost_summary"][
                        "Project summary"
                        ]["Total revenue (non-discounted)"]
                    .replace("$", "")
                    .replace(",", "")
                )
                - int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if breakeven_cost["project_params"][
                    "Project parameters"
                    ]["Carbon revenues to cover"] == "Opex"
                else int(
                    breakeven_cost["cost_summary"][
                        "Project summary"
                        ]["Total revenue (non-discounted)"]
                    .replace("$", "")
                    .replace(",", "")
                )
                - int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if breakeven_cost["project_params"][
                    "Project parameters"
                    ]["Carbon revenues to cover"] == "Opex"
                else int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
            "npv": {
                "total": int(
                    breakeven_cost["cost_summary"]["Project summary"]["Total revenue (NPV)"]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "leftover": int(
                    breakeven_cost["cost_summary"]["Project summary"]["Total revenue (NPV)"]
                    .replace("$", "")
                    .replace(",", "")
                )
                - int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if breakeven_cost["project_params"][
                    "Project parameters"
                    ]["Carbon revenues to cover"] == "Opex"
                else int(
                    breakeven_cost["cost_summary"]["Project summary"]["Total revenue (NPV)"]
                    .replace("$", "")
                    .replace(",", "")
                )
                - int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "opex": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                )
                if breakeven_cost["project_params"][
                    "Project parameters"
                    ]["Carbon revenues to cover"] == "Opex"
                else int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
        },
        "summary": {
            "$/tCO2e (total cost, NPV)": int(
                breakeven_cost["cost_summary"]["Project summary"]["$/tCO2e (total cost, NPV)"]
                .replace("$", "")
                .replace(",", "")
            ),
            "$/ha": int(breakeven_cost["cost_summary"][
                "Project summary"
                ]["$/ha"].replace("$", "").replace(",", "")),
            "NPV covering cost": int(
                breakeven_cost["cost_summary"]["Project summary"]["NPV covering cost"]
                .replace("$", "")
                .replace(",", "")
            ),
            # double check this attribute - this value should be the same as the NPV covering cost so how is it used?  # noqa: E501
            "Leftover after OpEx / total cost": int(
                breakeven_cost["cost_summary"]["Project summary"]["NPV covering cost"]
                .replace("$", "")
                .replace(",", "")
            ),
            "IRR when priced to cover OpEx": float(
                breakeven_cost["cost_summary"][
                    "Project summary"
                    ]["IRR when priced to cover opex"].replace("%", "")
            )
            / 100,
            "IRR when priced to cover total cost": float(
                breakeven_cost["cost_summary"][
                    "Project summary"
                    ]["IRR when priced to cover total costs"].replace(
                    "%", ""
                )
            )
            / 100,
            "Total cost (NPV)": int(
                breakeven_cost["cost_summary"][
                    "Project summary"
                    ]["Total cost (NPV)"].replace("$", "").replace(",", "")
            ),
            "Capital expenditure (NPV)": int(
                cost_summary["Project summary"][
                    "Capital expenditure (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Operating expenditure (NPV)": int(
                cost_summary["Project summary"][
                    "Operating expenditure (NPV)"
                    ].replace("$", "").replace(",", "")
            ),
            "Credits issued": int(
                breakeven_cost["cost_summary"]["Project summary"]["Credits issued"].replace(",", "")
            ),
            "Total revenue (NPV)": int(
                breakeven_cost["cost_summary"]["Project summary"]["Total revenue (NPV)"]
                .replace("$", "")
                .replace(",", "")
            ),
            "Total revenue (non-discounted)": int(
                breakeven_cost["cost_summary"]["Project summary"]["Total revenue (non-discounted)"]
                .replace("$", "")
                .replace(",", "")
            ),
            "Financing cost": int(
                breakeven_cost["cost_summary"][
                    "Project summary"
                    ]["Financing cost"].replace("$", "").replace(",", "")
            ),
            # dueble check - we should remove this and just show the fundin gap (NPV)
            "Funding gap": int(
                breakeven_cost["cost_summary"]["Project summary"]["Funding gap (NPV)"]
                .replace("$", "")
                .replace(",", "")
            ),
            "Funding gap (NPV)": int(
                breakeven_cost["cost_summary"]["Project summary"]["Funding gap (NPV)"]
                .replace("$", "")
                .replace(",", "")
            ),
            "Funding gap per tCO2e (NPV)": float(
                breakeven_cost["cost_summary"]["Project summary"]["Funding gap per tCO2e (NPV)"]
                .replace("$", "")
                .replace(",", "")
            ),
            "Community benefit sharing fund": int(
                breakeven_cost["cost_summary"][
                    "Project summary"
                    ]["Community benefit sharing fund % of revenue"]
                .replace("%", "")
                .replace(",", "")
            )
            / 100,
        },
        "costDetails": {
            "total": {
                "capitalExpenditure": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Capital expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "operationalExpenditure": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "totalCost": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "feasibilityAnalysis": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Feasibility analysis",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "conservationPlanningAndAdmin": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Conservation planning and admin",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "dataCollectionAndFieldCost": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Data collection and field costs",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityRepresentation": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Community representation / liaison",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "blueCarbonProjectPlanning": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Blue carbon project planning",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "establishingCarbonRights": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Establishing carbon rights",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "validation": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Validation",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "implementationLabor": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Implementation labor",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "monitoring": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Monitoring",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "maintenance": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Maintenance",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityBenefitSharingFund": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Community benefit sharing fund",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "carbonStandardFees": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Carbon standard fees",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "baselineReassessment": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Baseline reassessment",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "mrv": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "MRV",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "longTermProjectOperatingCost": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Long-term project operating",
                        "Total cost",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
            "npv": {
                "capitalExpenditure": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Capital expenditure",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "operationalExpenditure": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Operating expenditure",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "totalCost": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Total cost",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "feasibilityAnalysis": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Feasibility analysis",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "conservationPlanningAndAdmin": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Conservation planning and admin",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "dataCollectionAndFieldCost": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Data collection and field costs",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityRepresentation": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Community representation / liaison",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "blueCarbonProjectPlanning": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Blue carbon project planning",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "establishingCarbonRights": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Establishing carbon rights",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "validation": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Validation",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "implementationLabor": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Implementation labor",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "monitoring": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Monitoring",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "maintenance": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "Maintenance",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "communityBenefitSharingFund": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Community benefit sharing fund",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "carbonStandardFees": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Carbon standard fees",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "baselineReassessment": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Baseline reassessment",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "mrv": int(
                    breakeven_cost["cost_estimates"]
                    .loc[breakeven_cost["cost_estimates"]["Cost estimates (USD)"] == "MRV", "NPV"]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
                "longTermProjectOperatingCost": int(
                    breakeven_cost["cost_estimates"]
                    .loc[
                        breakeven_cost["cost_estimates"]["Cost estimates (USD)"]
                        == "Long-term project operating",
                        "NPV",
                    ]
                    .values[0]
                    .replace("$", "")
                    .replace(",", "")
                ),
            },
        },
        "yearlyBreakdown": [
            {
                "costName": "feasibilityAnalysis",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Feasibility analysis"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Feasibility analysis"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Feasibility analysis"][:-2].items()
                ),
            },
            {
                "costName": "conservationPlanningAndAdmin",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Conservation planning and admin"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Conservation planning and admin"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Conservation planning and admin"][:-2]
                    .items()
                ),
            },
            {
                "costName": "dataCollectionAndFieldCost",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Data collection and field costs"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Data collection and field costs"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Data collection and field costs"][:-2]
                    .items()
                ),
            },
            {
                "costName": "blueCarbonProjectPlanning",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Blue carbon project planning"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Blue carbon project planning"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Blue carbon project planning"][:-2]
                    .items()
                ),
            },
            {
                "costName": "communityRepresentation",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Community representation / liaison"][
                        "Total"
                    ]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Community representation / liaison"][
                        "NPV"
                    ]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Community representation / liaison"][:-2]
                    .items()
                ),
            },
            {
                "costName": "establishingCarbonRights",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Establishing carbon rights"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Establishing carbon rights"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Establishing carbon rights"][:-2].items()
                ),
            },
            {
                "costName": "validation",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Validation"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Validation"]["NPV"]),
                "costValues": dict(breakeven_cost["cost_pro_forma"].loc["Validation"][:-2].items()),
            },
            {
                "costName": "implementationLabor",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Implementation labor"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Implementation labor"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Implementation labor"][:-2].items()
                ),
            },
            {
                "costName": "monitoring",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Monitoring"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Monitoring"]["NPV"]),
                "costValues": dict(breakeven_cost["cost_pro_forma"].loc["Monitoring"][:-2].items()),
            },
            {
                "costName": "maintenance",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Maintenance"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Maintenance"]["NPV"]),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Maintenance"][:-2].items()
                ),
            },
            {
                "costName": "communityBenefitSharingFund",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Community benefit sharing fund"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Community benefit sharing fund"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Community benefit sharing fund"][:-2]
                    .items()
                ),
            },
            {
                "costName": "carbonStandardFees",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Carbon standard fees"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Carbon standard fees"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Carbon standard fees"][:-2].items()
                ),
            },
            {
                "costName": "baselineReassessment",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Baseline reassessment"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Baseline reassessment"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Baseline reassessment"][:-2].items()
                ),
            },
            {
                "costName": "mrv",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["MRV"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["MRV"]["NPV"]),
                "costValues": dict(breakeven_cost["cost_pro_forma"].loc["MRV"][:-2].items()),
            },
            {
                "costName": "longTermProjectOperatingCost",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Long-term project operating"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Long-term project operating"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Long-term project operating"][:-2].items()
                ),
            },
            {
                "costName": "opexTotalCostPlan",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Total opex"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Total opex"]["NPV"]),
                "costValues": dict(breakeven_cost["cost_pro_forma"].loc["Total opex"][:-2].items()),
            },
            {
                "costName": "capexTotalCostPlan",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Total capex"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Total capex"]["NPV"]),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Total capex"][:-2].items()
                ),
            },
            {
                "costName": "totalCostPlan",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Total cost"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Total cost"]["NPV"]),
                "costValues": dict(breakeven_cost["cost_pro_forma"].loc["Total cost"][:-2].items()),
            },
            {
                "costName": "estimatedRevenuePlan",
                "totalCost": float(breakeven_cost["cost_pro_forma"].loc["Est. revenue"]["Total"]),
                "totalNPV": float(breakeven_cost["cost_pro_forma"].loc["Est. revenue"]["NPV"]),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Est. revenue"][:-2].items()
                ),
            },
            {
                "costName": "creditsIssuedPlan",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Est. credits issued"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Est. credits issued"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Est. credits issued"][:-2].items()
                ),
            },
            {
                "costName": "cumulativeNetIncomePlan",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Cumulative net income (revenue - OPEX)"][
                        "Total"
                    ]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Cumulative net income (revenue - OPEX)"][
                        "NPV"
                    ]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Cumulative net income (revenue - OPEX)"][:-2]
                    .items()
                ),
            },
            {
                "costName": "cumulativeNetIncomeCapexOpex",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc[
                        "Cumulative net income (revenue - CAPEX - OPEX)"
                    ]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc[
                        "Cumulative net income (revenue - CAPEX - OPEX)"
                    ]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Cumulative net income (revenue - CAPEX - OPEX)"][:-2]
                    .items()
                ),
            },
            {  # sum of total cost + est revenue
                "costName": "annualNetCashFlow",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Annual net cash flow"]["Total"]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Annual net cash flow"]["NPV"]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"].loc["Annual net cash flow"][:-2].items()
                ),
            },
            {  # sum total opex + est revenue
                "costName": "annualNetIncome",
                "totalCost": float(
                    breakeven_cost["cost_pro_forma"].loc["Annual net income (revenue – OPEX)"][
                        "Total"
                    ]
                ),
                "totalNPV": float(
                    breakeven_cost["cost_pro_forma"].loc["Annual net income (revenue – OPEX)"][
                        "NPV"
                    ]
                ),
                "costValues": dict(
                    breakeven_cost["cost_pro_forma"]
                    .loc["Annual net income (revenue – OPEX)"][:-2]
                    .items()
                ),
            },
        ],
    },
}

output

# export output as json file
with open(f"../test_data/output_{project_name}.json", "w") as f:
    json.dump(output, f)